# Plan for today

1.  **Aggregation and reduction**

    -   Performing computations on (subset of) data

2.  **Combining data**

    -   Concatenation
    -   Merging / joining
    -   Handling missing values

***
# Environment setup

In [ ]:
# Uncomment this to use files in the local data/ directory
DATA_PATH = '../../data'

# Uncomment this to load data directly from GitHub
# DATA_PATH = 'https://raw.githubusercontent.com/richardfoltyn/FIE463-V25/main/data'

In [ ]:
import pandas as pd 

# Display only 6 rows of data
pd.set_option('display.max_rows', 6)

***
# Grouping and aggregation with pandas

## Aggregation and reduction

*Split-apply-combine* operations:

1. *Split* data into groups based on some criteria;
2. *Apply* some function to each group separately; and
3. *Combine* the results into a single `DataFrame` or `Series`.

See [cheat sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) for an illustration of such operations.

***
### Aggregations of whole Series or DataFrames

Pandas supports the usual set of aggregation functions, e.g.:

-   [`mean()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html)
-   [`sum()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html)
-   [`std()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.std.html), 
    [`var()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.var.html)
-   [`quantile()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html)
-   [`count()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html)
-   [`min()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.min.html), 
    [`max()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.max.html)

*Example: compute mean of all numerical columns*

In [ ]:
import pandas as pd

# Path to Titanic passenger data CSV file
file = f'{DATA_PATH}/titanic.csv'

# Read in Titanic passenger data, set PassenderId column as index
df = pd.read_csv(file, index_col='PassengerId')

*Example: NaNs are automatically ignored (unlike with NumPy)*

***
### Aggregations of subsets of data (grouping)

-   Apply aggregation by group
    -   Groups are defined based on values of columns or the index
    -   Usually a categorical variable or an identifier for a household/individual/firm/period/...
    
-   Can group by *multiple* columns or index levels at once

*Example: apply groupby-operations to SCF data*

-   We use a 10% sample of the
    [Survey of Consumer Finances](https://www.federalreserve.gov/econres/scfindex.htm) 
    (SCF) for the years 1989-2022
-   Use [`groupby()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)
    to group by employment state (column `empl`):

    1. working for someone else
    2. self-employed
    3. retired/disabled
    4. not in labor force

In [ ]:
# Path to SCF data file
file = f'{DATA_PATH}/SCF/SCF_10pct.csv'

# Read in SCF data, set id column as index
df = pd.read_csv(file, index_col='id')

#### Built-in aggregations

- [`mean()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.mean.html):
    averages within each group
- [`sum()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.sum.html):
    sum values within each group
- [`std()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.std.html), 
    [`var()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.var.html): 
    within-group standard deviation and variance
-   [`median()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.median.html):
    compute median within each group
- [`quantile()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.quantile.html):
    compute quantiles within each group
- [`size()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.size.html): 
    number of observations in each group
- [`count()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.count.html):
    number of non-missing observations in each group
- [`first()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.first.html), 
    [`last()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.last.html): 
    first and last elements in each group
-   [`min()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.min.html), 
    [`max()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.max.html): 
    minimum and maximum elements within a group

See the [official documentation](https://pandas.pydata.org/docs/user_guide/groupby.html#built-in-aggregation-methods) for a complete list.

*Example: Number of elements within each group*

-   Note: `size()` and `count()` are two different functions!

*Example: Return first observation of each group*

<div class="alert alert-info">
<h3> Your turn</h3>
Use the SCF data set to perform the following aggregations:
<ol>
    <li>Compute the average net worth (<TT>networth</TT>) by marital status (<TT>married</TT>).</li>
    <li>Compute the median value of the primary residence (<TT>houses</TT>) by education (<TT>educ</TT>).</li>
    <li>Compute the home ownership rate (<TT>owner</TT>) by marital status (<TT>married</TT>) <i>and</i> the sex of the household head (<TT>female</TT>).</li>
</ol>
</div>

***
#### Writing custom aggregations

-   Sometimes we want to use aggregations *not* implemented in pandas
-   Use [`agg()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html)
    to perform custom aggregations
-   Important: `agg()` operates on a single column at a time (cannot combine data from multiple columns)

    -   Use [`apply()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.apply.html) to perform aggregations based on multiple columns

*Example: compute mean via `agg()`*

-   Compute mean age by employment status

*Example: custom aggregation*

-   Count number of households with a household head older than 60 years by employment state

***
#### Applying multiple functions at once

-  Applying multiple functions to a **single** column

    -   Functions are passed in as a *list*

-  Applying multiple functions to **multiple** columns

    -   Use so-called *named aggregation*
    -   Columns and functions are passed in as a *dictionary*:

    ```python
    groups.agg(
        new_column_name1=('column_name1', 'operation1'),
        new_column_name2=('column_name2', 'operation2'),
        ...
    )
    ```


*Example: Applying multiple functions to a **single** column*

-   Compute mean *and* median age by employment status

*Example: Applying multiple functions to **multiple** columns*

-   Compute the following by employment status in a single call to `agg()`:
    1.  Median age
    2.  Smallest net worth
    3.  Share of female household heads

<div class="alert alert-info">
<h3> Your turn</h3>
Use the SCF data set to perform the following aggregations:
<ol>
    <li>Compute the minimum, maximum and average age (<TT>age</TT>) by marital status <i>and</i> sex (<TT>married</TT> and <TT>female</TT>) in a single <TT>agg()</TT> operation.</li>
    <li>Compute the number of observations, the home ownership rate (<TT>owner</TT>), and median net worth (<TT>networth</TT>) by education level (<TT>educ</TT>) in a single <TT>agg()</TT> operation.
</ol>
</div>

***
## Transformations

-   Same principle as aggregations, but size of the result object remains unchanged
-   Useful for computations that involve individual and aggregate data
-   Performed with [`transform()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.transform.html)

*Example: assign group-specific mean*

- Compute home ownership rate by eduction, store as `frac_owners` column

*Example: Deviation from median net worth*

-   For each household, compute difference to median net worth of households with the same education

<div class="alert alert-info">
<h3> Your turn</h3>
Use the SCF data set to answer the following questions:
<ol>
    <li>Compute how much a household pays more in rent (<TT>rent</TT>) than the average household with the same 
    marital (<TT>married</TT>) and employment status (<TT>empl</TT>). Restrict your analysis
    to households who do not own their residence (<TT>owner = 0</TT>).</li>
</ol>
</div>

***
## Resampling and aggregation

-   Use [`resample()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html) to aggregate time series data to lower frequency
-   Argument to `resample()` determines frequency and index of resulting data, e.g.:

    -   `'YE'` aggregation to years, index is end of year
    -   `'QE'` aggregation to quarters, index is end of quarter
    -   `'ME'` aggregation to months, index is end of month
    -   `'W'` aggregations to weeks

*Example: Resampling the NASDAQ index*

In [ ]:
# Path to NASDAQ data file
file = f'{DATA_PATH}/stockmarket/NASDAQ.csv'

# Read in NASDAQ data, set Date column as index
df = pd.read_csv(file, index_col='Date', parse_dates=True)

# Keep observations after 2024
df = df.loc['2024':]

<div class="alert alert-info">
<h3> Your turn</h3>
<p>
Use the daily NASDAQ data for 2024 and compute the percentage change from the first to the last trading day within each month.
</p>
</div>

***
# Concatenating and merging data

Combine several `Series` or `DataFrame` objects:

1. [`pd.concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html): combine multiple Series/DataFrames by appending observations (rows) or columns.
2. [`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html): match observations from one Series/DataFrame with observations from another Series/DataFrame and combine these into a _merged_ DataFrame.

## Concatenation

### Concatenating Series

In [ ]:
# Create demo series
import pandas as pd 

a = pd.Series(['A1', 'A2', 'A3'])
b = pd.Series([f'B{i}' for i in range(5)])

*Example: Concatenating two Series along the row axis*

-   Might need to reset index of result object

*Example: Concatenating along the column axis*

-   Need to specify `axis=1` argument
-   Use `keys` to specify column names

<div class="alert alert-info">
<h3> Your turn</h3>
<ol>
    <li>Create a new <TT>Series</TT> with observations <TT>['C1', 'C2']</TT>.</li>
    <li>Using the previously created <TT>Series</TT> <TT>a</TT> and <TT>b</TT>, concatenate all three objects along the row axis and create a new (unique) index.</li>
    <li>Repeat the previous step, but now concatenate along the column axis. Assign the column names <TT>'Column1'</TT>, <TT>'Column2'</TT>, and <TT>'Column3'</TT>.</li>
</ol>
</div>

### Concatenating DataFrames

In [ ]:
# Create demo DataFrames

import pandas as pd
import numpy as np

# Create 2 x 2 DataFrame of string data
df_a = pd.DataFrame(np.array(('A1', 'A2', 'A3', 'A4')).reshape((2, 2)))

# Create 2 x 3 DataFrame of string data
df_b = pd.DataFrame(np.array([f'B{i}' for i in range(6)]).reshape((2, 3)))

#### Concatenating along the column axis

*Example: Concatenating two DataFrames along the column axis*

-   Might need to reset non-unique column index
-   Use `keys` argument to `concat()` to add outer level to column index

#### Concatenating along the row axis

*Example: Concatenating rows with identical columns*

*Example: Concatenating rows with different columns*

<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>FRED_monthly_1950.csv</TT> and <TT>FRED_monthly_1960.csv</TT> into two different DataFrames.
        The files contain monthly macroeconomic time series for the 1950s and 1960s, respectively.
        <p>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </p>
        </li>
    <li>Concatenate these DataFrames along the row dimension to get a total of 240 observations.</li>
    <li>Set the column <TT>DATE</TT> as index for the newly created DataFrame.</li>
</ol>
</div>

In [ ]:
# Read in FRED data for 1950s and 1960s
df_1950 = pd.read_csv(f'{DATA_PATH}/FRED/FRED_monthly_1950.csv', parse_dates=['DATE'])
df_1960 = pd.read_csv(f'{DATA_PATH}/FRED/FRED_monthly_1960.csv', parse_dates=['DATE'])

***
## Merging and joining data sets

### Types of merges

We can merge DataFrames `A` and `B` in various ways:

1.  *one-to-one*: Unique keys in `A` and `B`
2.  *many-to-one*: Unique keys in `A`, non-unique keys in `B`
3.  *many-to-many*: Non-unique keys in both `A` and `B` (avoid this)

### Implementation in pandas

1.  [`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html): function that takes as argument the *two* DataFrames to be merged,
    e.g.,
    
    ```python
    result = pd.merge(df_A, df_B)
    ```
2.  [`df.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html): method of a specific `DataFrame` object, takes the other `DataFrame` as argument, e.g.,
    
    ```python
    result = df_A.merge(df_B)
    ```


### Controlling the resulting data set

Merge methods:

1.  `how='inner'` (_inner join_): _intersection_ of keys that are present in _both_ data sets
2.  `how='outer'` (_outer join_): _union_ of keys present in either of the data sets
3.  `how='left'` (_left join_): all identifiers from the _left_ data set are in the result
4.  `how='right'` (_right join_): all identifiers from the _right_ data set are in the result

Illustration: Each circle represents the keys present in the left (`df1`) or right (`df2`) DataFrames. The merge method controls which subset of keys is retained in the merge result.

![Join types](join-methods.png)

***
### Merging with `merge()`

In [ ]:
# Create first DataFrame with 2 rows
df_a = pd.DataFrame({'key': [0, 1], 'value_a': ['A0', 'A1']})

# Create second DataFrame with 2 rows
df_b = pd.DataFrame({'key': [1, 2], 'value_b': ['B1', 'B2']})

#### Using `pd.merge()`

*Example: one-to-one merges*

-   Argument `on` controls on which columns to match (default: use all overlapping columns)
-   Argument `how` controls which rows to keep in result

#### Using `DataFrame.merge()`

-   Equivalent to `pd.merge()`

*Example: Merging with overlapping column names*

In [ ]:
# Rename columns to common name 'value'
df_a = df_a.rename(columns={'value_a': 'value'})
df_b = df_b.rename(columns={'value_b': 'value'})

<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <p>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </p>
        </li>
    <li>Merge the CPI with the GDP time series with 
    <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html"><TT>merge()</TT></a> 
    using a left join (<TT>how='left'</TT>). How many observations does the resulting DataFrame have?</li>
    <li>Merge the CPI with the GDP time series with <TT>merge()</TT> using an inner join (<TT>how='inner'</TT>). How many observations does the resulting DataFrame have,
        and why is this different from the previous case?</li>
</ol>
</div>

In [ ]:
# Read in CPI and GDP data
df_cpi = pd.read_csv(f'{DATA_PATH}/FRED/CPI.csv', parse_dates=['DATE'])
df_gdp = pd.read_csv(f'{DATA_PATH}/FRED/GDP.csv', parse_dates=['DATE'])

***
### Joining with `join()`

The `DataFrame` method 
[`join()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html) 
is a convenience wrapper around 
[`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html):

1.  `join()` can be called _only_ directly on the `DataFrame` object
2.  `join()` always operates on the _index_ of the other `DataFrame`
3.  `join()` by default performs a `left` join, whereas `merge()` performs an `inner` join

Use `join()` if you want to join DataFrames which have a similar index.

*Example: joining DataFrames*

In [ ]:
# Create first DataFrame with 2 rows
df_a = pd.DataFrame(['A0', 'A1'], columns=['value_a'], index=[0, 1])

# Create second DataFrame with 2 rows
df_b = pd.DataFrame(['B1', 'B2'], columns=['value_b'], index=[1, 2])

<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <br/>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </li>
    <li>Set the <TT>DATE</TT> column as the index for each of the two DataFrames.</li>
    <li>Merge the CPI with the GDP time series with 
    <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html"><TT>join()</TT></a>. 
    Do this with both a left and an inner join.</li>
</ol>
</div>

***
# Dealing with missing values

-   Often created as result of `pd.merge()` or `pd.concat()` if operands don't have the same row or column index

In [ ]:
# Create two DataFrames with partially overlapping keys
df_a = pd.DataFrame({'key': [0, 1], 'value_a': ['A0', 'A1']})
df_b = pd.DataFrame({'key': [1, 2], 'value_b': ['B1', 'B2']})

## Dropping missing values

Missing values can be dropped by either

1. Using [`dropna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)
    or selecting a subset of observations with a boolean operation such as 
    [`notna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.notna.html).
2. Avoiding the missing values in the first place, e.g., by using `merge(..., how='inner')`.

*Example: Dropping missing values*

- Use _outer/left/right join_ and drop missing data with `dropna()`

*Example: Avoiding missing values in the first place*

- E.g., use _inner join_ when merging data sets

## Filling missing values

Instead of dropping data, we can impute missing values in various ways:

1.  [`fillna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html): replace missing data with user-specified values
2.  [`ffill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html) and 
    [`bfill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bfill.html): fill missing values
    forward or backward from adjacent non-missing observations
3.  [`interpolate()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html): interpolate missing values based on non-missing ones

*Example: Replacing missing values with `fillna()`*

- Specify one value for _entire_ `DataFrame`
- Specify different value for each column using a dictionary

*Example: forward- or backward-filling missing values*

- Use [`ffill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html) to forward-fill
- Use [`bfill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bfill.html) to backward-fill

*Example: linear interpolation*

- Use [`interpolate()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.interpolate.html) to interpolate missing _numerical_ data (e.g., `method='linear'`)

In [ ]:
s = pd.Series([1.0, 2.0, 3.0, np.nan, 5.0])

<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <br/>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </li>
    <li>Merge the CPI with the GDP time series with <TT>merge()</TT> using a left join. This creates missing values in the <TT>GDP</TT>
    column.</li>
    <li>Impute the missing GDP values using <a href="https://pandas.pydata.org/docs/reference/api/pandas.Series.interpolate.html"><TT>interpolate()</TT></a> 
    and replace the missing values in column <TT>GDP</TT>.</li>
</ol>
</div>